Physics 215 (1st Semester AY 2022-2023)

Richelle Jade L. Tuquero

# Session 3 Types, type inference and stability

**OBJECTIVE**: Demonstrate the dynamic programming features of Julia
* [X] KR1: Shown or demonstrated the hierarchy of Julia’s type hierarchy using the command ``subtypes()``. Start from Number and use ``subtypes()`` to explore from  down to specific types. Use ``supertype()`` to determine the  abstract type.
* [X] KR2: Implemented and used at least one own composite type via ``struct``. Generate two more versions that are mutable type and type-parametrized of the custom-built type.
* [X] KR3: Demonstrated type inference in Julia. Generator expressions may be used for this.
* [X] KR4: Created a function with inherent type-instability. Create a version of the function with fixed  issues.
* [X] KR5: Demonstration of how ``@code_warntype`` can be useful in detecting type-instabilities.
* [X] KR6: Demonstration of how Arrays containing ambiguous/abstract types often results to slow execution of codes. The ``BenchmarkTools`` may be useful in this part.

In [1]:
using Pkg
Pkg.activate(".")
Pkg.add("BenchmarkTools")
Pkg.update()
Pkg.status()

  Activating project at `~/Desktop/Physics 215/Submission/Session 3`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
  No Changes to `~/Desktop/Physics 215/Submission/Session 3/Project.toml`
  No Changes to `~/Desktop/Physics 215/Submission/Session 3/Manifest.toml`
    Updating registry at `~/.julia/registries/General.toml`
  No Changes to `~/Desktop/Physics 215/Submission/Session 3/Project.toml`
  No Changes to `~/Desktop/Physics 215/Submission/Session 3/Manifest.toml`


Status `~/Desktop/Physics 215/Submission/Session 3/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [91a5bcdd] Plots v1.36.1


# KR1
Shown or demonstrated the hierarchy of Julia’s type hierarchy using the command ``subtypes()``. Start from Number and use ``subtypes()`` to explore from down to specific types. Use ``supertype()`` to determine the abstract type.

We first check the result of ``subtypes()`` for ``Number`` and its subtypes.

In [2]:
subtypes(Number)

2-element Vector{Any}:
 Complex
 Real

In [3]:
subtypes(Real)

4-element Vector{Any}:
 AbstractFloat
 AbstractIrrational
 Integer
 Rational

In [4]:
subtypes(Complex)

Type[]

We observe that the ``Number`` type consists of Complex and Real numbers. However, upon further investigation we notice that the ``Real`` type consists of other subtypes while the ``Complex`` type has none. From this, we decided to create a function that will show us the hierarchy of the types starting from ``Number``. We refer to this function as ``det_subtype``.

In [5]:
? det_subtype

search:

Couldn't find det_subtype
Perhaps you meant subtypes


No documentation found.

Binding `det_subtype` does not exist.


Since there is no function by that name, we start defining the function ``det_subtype`` which will determine the subtypes starting from the type ``var``. To get a better look at the hierarchy, we include indentations given by the string ``"...|"``. Moreover, since a subtype might contain its own subtype then we also call the function inside the same function for that specific type.

In [6]:
"""
    def_subtype( var; hierar = 1 )
Determines the subtype of var and the other specific subtypes.
- Input: ``var`` initial data type
        ``hierar`` states the level.
"""
function def_subtype( var; hierar = 1)
    level = "...|"
    type_var = subtypes( var )    # Determine initial subtype/s
    if hierar == 1                # For spacing
        println( var)
    end
    for i in type_var             # For-loop to check the subtypes of each subtypes
        println(string(hierar)*level^hierar * string(i))
        def_subtype(i; hierar = hierar+1)
    end
end

def_subtype

In [7]:
def_subtype(Number)

Number
1...|Complex
1...|Real
2...|...|AbstractFloat
3...|...|...|BigFloat
3...|...|...|Float16
3...|...|...|Float32
3...|...|...|Float64
2...|...|AbstractIrrational
3...|...|...|Irrational
2...|...|Integer
3...|...|...|Bool
3...|...|...|Signed
4...|...|...|...|BigInt
4...|...|...|...|Int128
4...|...|...|...|Int16
4...|...|...|...|Int32
4...|...|...|...|Int64
4...|...|...|...|Int8
3...|...|...|Unsigned
4...|...|...|...|UInt128
4...|...|...|...|UInt16
4...|...|...|...|UInt32
4...|...|...|...|UInt64
4...|...|...|...|UInt8
2...|...|Rational


In here, we observe that the subtypes of ``Number`` are ``Complex`` and ``Real`` where ``Real`` contains more specific subtypes and ``Complex`` has none. To get a better idea, we check the type of ``Complex`` and ``Real`` using ``typeof``.

In [8]:
typeof(Complex)

UnionAll

In [9]:
typeof(Real)

DataType

This confirms that ``Complex`` has no subtypes unlike ``Real`` since ``Complex`` is not a ``DataType`` and instead it is a ``UnionAll``. This makes sense since we have ``Complex`` by including ``im`` to a real number making it a union of types. Hence, ``Complex`` is a concrete type since it does not have a subtype, unlike ``Real`` which is an abstract type.

For a more general discussion, the abstract types are ``Number``, ``Complex``, ``Real``, ``AbstractFloat``, ``AbstractIrrational``, ``Integer``, ``Signed``, and ``Unsigned``. These does not hold any value and are characterized by having subtypes as shown in the hierarchy. Concrete types on the otherhand are ``Complex``, ``BigFloat``, ``Float16``, ``Float32``, ``Float64``, ``Irrational``, ``Bool``, ``BigInt``, ``Int128``, ``Int16``, ``Int32``, ``Int64``, ``Int8``, ``UInt128``, ``UInt16``, ``UInt32``, ``UInt64``, ``UInt8``, and ``Rational``. These are types which can hold values.

It is important to note the difference between these concrete types, specifically those with abstract types ``AbstarctFloat``, ``Signed``, and ``Unsigned``, are commonly due to the number of bits used for the stored value. Other interesting differences is that the ``Rational`` type can hold values as fractions by having integers in the numerator and denominator, separated by ``//``. Next, the ``Irrational`` type refers to the constants like $\pi$ and $e$.

Then, we consider determining the abstract type by using ``supertype()``. We first try it for ``Union``.

In [10]:
supertype(Union)

Type{T}

In [11]:
supertype(supertype(Union))

Any

Hence, we confirm that ``Union`` refers to the bottom of the hierarchy which is present in Any types. This means that all the types can be the abstract type of ``Union``. Next let us check another type which is ``Int8``.

In [12]:
supertype(Int8)

Signed

In [13]:
supertype(supertype(Int8))

Integer

In [14]:
supertype(supertype(supertype(Int8)))

Real

Notice that the supertype or abstract type of ``Int8`` is ``Signed`` which has the abstract type of ``Integer`` then ``Real``, and finally ``Number``. While we can check the respective abstract type, it would take time to check it manually. Hence, we define a function called ``def_supertype`` to check the supertype until ``Number`` for a specified type ``var``.

In [15]:
? def_supertype

search:

Couldn't find def_supertype
Perhaps you meant def_subtype or supertype


No documentation found.

Binding `def_supertype` does not exist.


In [16]:
"""
    def_supertype( var; hierar = 1 )
Determines the subtype of var and the other specific subtypes.
- Input: ``var`` initial data type
        ``hierar`` states the level of the abstract type.
"""
function def_supertype( var; hierar = 1)
    level = "...|"
    type_var = supertype( var )      # Determines the supertype
    if hierar == 1                   # For initial spacing
        println( var)
    end
    println("-"*string(hierar)*level^hierar * string(type_var))
    if type_var != Number && type_var != Any     # Determine the subtype until it reaches Number and Any
        def_supertype(type_var; hierar = hierar+1)
    end
end

def_supertype

In [17]:
def_supertype(Int8)

Int8
-1...|Signed
-2...|...|Integer
-3...|...|...|Real
-4...|...|...|...|Number


In [18]:
def_supertype(Union)

Union
-1...|Type{T}
-2...|...|Any


In [19]:
def_supertype(Irrational)

Irrational
-1...|AbstractIrrational
-2...|...|Real
-3...|...|...|Number


# KR2
Implemented and used at least one own composite type via struct. Generate two more versions that are mutable type and type-parametrized of the custom-built type.

In this section, we consider using different types of ``struct``. ``struct`` are structured data that contains the information of a specific object. This is comparable to classes in other programming languages. In here, we choose to create a ``struct`` called as ``wavefunction``. This contains the number of particles ``N``, the amplitudes or constants ``amp`` for each ``state`` ``Vector``.

In [20]:
struct wavefunction
    N::Real        # Number of particles
    amp::Vector    # Constants
    state::Vector  # States
end

If we want to refer to the ``wavefunction`` using other names, for example ``wavef``. Then, we can do this by doing the following.

In [21]:
wavef = wavefunction
println(typeof(wavef))

DataType


Let us try to introduce the values for the variables for the wavefunction or ``wavef``.

In [22]:
N = 1;
amp = [1/sqrt(2), 1/sqrt(2)];
state = [[1;0], [0;1]];

In [23]:
psi = wavef(N, amp, state)
println("typeof(psi) = $(typeof(psi))")
println("psi = $(psi)")
println("psi.N = $(psi.N)")
println("psi.amp = $(psi.amp)")
println("psi.state = $(psi.state)")

typeof(psi) = wavefunction
psi = wavefunction(1, [0.7071067811865475, 0.7071067811865475], [[1, 0], [0, 1]])
psi.N = 1
psi.amp = [0.7071067811865475, 0.7071067811865475]
psi.state = [[1, 0], [0, 1]]


Next, we try to change the values in the vectors included in the ``struct``.

In [24]:
psi.amp[:] = [1, 0]

2-element Vector{Int64}:
 1
 0

In [25]:
psi.N = 100

LoadError: setfield!: immutable struct of type wavefunction cannot be changed

From the above result it shows that we can only change the reference values of a vector included in the ``struct`` with specified index. However, if we were to change the properties like ``N`` we are reminded that the ``struct`` is immutable and as such we cannot reassign a value for the property.

Next, we consider creating a mutable struct version of the previous struct.

In [26]:
mutable struct Mwavefunction
    N::Real        # Number of particles
    amp::Vector    # Constants
    state::Vector  # States
end

In [27]:
mpsi = Mwavefunction(N, amp, state)

Mwavefunction(1, [1.0, 0.0], [[1, 0], [0, 1]])

Let us try if we can change a reference value by reassigning the value of ``N`` again and check if we can instead input it as another abstract type like ``complex``.

In [28]:
mpsi.N = 100

100

In [29]:
mpsi.N = 100 - 1im

LoadError: InexactError: Real(100 - 1im)

From the ``mutable struct``, we can change the values of the reference inputs. While this is helpful, this might be prone to error since the properties can be changed. However, it is also important to note that the data type cannot be changed. In the example, we changed ``N`` to ``complex`` even if it should be ``Real``. Thus, resulting to an ``InexactError``.

Next, we consider parametrization of the type in the struct.

In [30]:
struct Pwavefunction{T}
    N::T       # Number of particles
    amp::Vector    # Constants
    state::Vector{Vector{T}}  # States
end

In [31]:
ppsi = Pwavefunction(N, amp, state)

Pwavefunction{Int64}(1, [1.0, 0.0], [[1, 0], [0, 1]])

By parametrizing the types of the ``composite types`` we are able to assign a more varied and flexible data types for the reference values. While Julia can indeed do this without parametrization, it would take longer to finish running as compared to parametrizing the types. Do note that this still makes the reference values immutable.

# KR3
Demonstrated type inference in Julia. Generator expressions may be used for this.

In [32]:
[i for i in 1:10]

10-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10

In [33]:
[i//2 for i in 1:10]

10-element Vector{Rational{Int64}}:
 1//2
 1//1
 3//2
 2//1
 5//2
 3//1
 7//2
 4//1
 9//2
 5//1

In [34]:
[i/2 for i in 1:10]

10-element Vector{Float64}:
 0.5
 1.0
 1.5
 2.0
 2.5
 3.0
 3.5
 4.0
 4.5
 5.0

As shown, Julia like Python can immediately determine the data type of the expression. The data type that is inferred by Julia depends on the data type of the inputs as well as the expression or operations used. In the example that we showed, we did it for an ``Int64`` inputs which gave us a ``Int64`` output. However, if the expression that is calculated does not follow the data type of the input like expressing this as fractions which gave a ``Rational`` output with ``Int64`` numerator and denominator. The other example is by dividing it which will result to a data type output of ``Float64``.

# KR4 and KR5
In this section, we include KR4 and KR5 to show what are the sources of type-instabilities.

**KR4**: Created a function with inherent type-instability. Create a version of the function with fixed  issues.

**KR5**: Demonstration of how ``@code_warntype`` can be useful in detecting type-instabilities.


We define a function for Arithmetic series or progression given by
\begin{equation}\label{1}
a_n = a_1 + (n-1)d.
\end{equation}
But instead of using Eq. \eqref{1}, we will implement a for-loop instead to demonstrate type-instability.

In [35]:
"""
    arith_series(a1, d, n)
Determines and returns the nth order in the arithmetic series.
- Input: ``a1`` initial value
         ``d``  distance or difference of two succedding values.
         ``n``  nth order
"""
function arith_series(a1, d, n)
    sum = a1    # initial value
    for i in range(0, n)
            sum += d
    end
    return sum
end

arith_series

Next, we demonstrate that depending on the data type of the input values, the data type for the return values are different.

In [36]:
println("typeof(arith_series(1, 2, 4)) = $(typeof(arith_series(1, 2, 4))).")
println("typeof(arith_series(1, 2.0, 4)) = $(typeof(arith_series(1.0, 2.0, 4))).")
println("typeof(arith_series(1.0, 2.0, 4)) = $(typeof(arith_series(1.0, 2.0, 4))).")
println("typeof(arith_series(1.0, 2.0, 4.0)) = $(typeof(arith_series(1.0, 2.0, 4.0))).")

typeof(arith_series(1, 2, 4)) = Int64.
typeof(arith_series(1, 2.0, 4)) = Float64.
typeof(arith_series(1.0, 2.0, 4)) = Float64.
typeof(arith_series(1.0, 2.0, 4.0)) = Float64.


We note that regardless if the values are the same, the data type would still be different when the data type of the inputs are different. An example that we implemented is ``1`` and ``2`` which are ``Int64``. If one of the input data types is ``Float64`` then the data type for the return value would be ``Float64``. Moreover, if we have multiple data types present in the code, then this is also an example of type-instability.

In [37]:
"""
    comp_arith_series(a1, d, n, comp)
Determines and returns which is higher between comp and the nth order in the arithmetic series.
- Input: ``a1`` initial value
         ``d``  distance or difference of two succedding values.
         ``n``  nth order
         ``comp`` the compared number
"""
function comp_arith_series(a1, d, n, comp)
    sum = arith_series(a1, d, n)   # returns the result for arithmetic series at the nth order.
    # Comparing which is higher and returning the higher value.
    if sum >= comp     
        return sum
    else
        return comp
    end
end

comp_arith_series

In [38]:
println("typeof(comp_arith_series(1, 2, 4, 20)) = $(typeof(comp_arith_series(1, 2, 4, 20))).")
println("typeof(comp_arith_series(1, 2.0, 4, 10)) = $(typeof(comp_arith_series(1.0, 2.0, 4, 10))).")
println("typeof(comp_arith_series(1.0, 2.0, 4, 20)) = $(typeof(comp_arith_series(1.0, 2.0, 4, 20))).")
println("typeof(comp_arith_series(1.0, 2.0, 4.0, 20.0)) = $(typeof(comp_arith_series(1.0, 2.0, 4.0, 10.0))).")

typeof(comp_arith_series(1, 2, 4, 20)) = Int64.
typeof(comp_arith_series(1, 2.0, 4, 10)) = Float64.
typeof(comp_arith_series(1.0, 2.0, 4, 20)) = Int64.
typeof(comp_arith_series(1.0, 2.0, 4.0, 20.0)) = Float64.


Next, we showed another reason why there is instability and this is when the possible return values have different data types.

To get a better idea on what are the sources of type-instabilities that we should focus on. We use the ``@code_warntype`` to help us determine what parts or line of the code does this type-instabilities come from. We first check for ``arith_series`` then ``comp_arith_series``.

In [39]:
@code_warntype arith_series(1, 2, 4)

MethodInstance for arith_series(::Int64, ::Int64, ::Int64)
  from arith_series(a1, d, n) in Main at In[35]:8
Arguments
  #self#::Core.Const(arith_series)
  a1::Int64
  d::Int64
  n::Int64
Locals
  @_5::Union{Nothing, Tuple{Int64, Int64}}
  sum::Int64
  i::Int64
Body::Int64
1 ─       (sum = a1)
│   %2  = Main.range(0, n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(0), Int64])
│         (@_5 = Base.iterate(%2))
│   %4  = (@_5 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_5::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (sum = sum + d)
│         (@_5 = Base.iterate(%2, %9))
│   %12 = (@_5 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return sum



In [40]:
@code_warntype arith_series(1, 2.0, 4)

MethodInstance for arith_series(::Int64, ::Float64, ::Int64)
  from arith_series(a1, d, n) in Main at In[35]:8
Arguments
  #self#::Core.Const(arith_series)
  a1::Int64
  d::Float64
  n::Int64
Locals
  @_5::Union{Nothing, Tuple{Int64, Int64}}
  sum::Union{Float64, Int64}
  i::Int64
Body::Union{Float64, Int64}
1 ─       (sum = a1)
│   %2  = Main.range(0, n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(0), Int64])
│         (@_5 = Base.iterate(%2))
│   %4  = (@_5 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_5::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (sum = sum + d)
│         (@_5 = Base.iterate(%2, %9))
│   %12 = (@_5 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return sum



In [41]:
@code_warntype arith_series(1.0, 2.0, 4)

MethodInstance for arith_series(::Float64, ::Float64, ::Int64)
  from arith_series(a1, d, n) in Main at In[35]:8
Arguments
  #self#::Core.Const(arith_series)
  a1::Float64
  d::Float64
  n::Int64
Locals
  @_5::Union{Nothing, Tuple{Int64, Int64}}
  sum::Float64
  i::Int64
Body::Float64
1 ─       (sum = a1)
│   %2  = Main.range(0, n)::Core.PartialStruct(UnitRange{Int64}, Any[Core.Const(0), Int64])
│         (@_5 = Base.iterate(%2))
│   %4  = (@_5 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_5::Tuple{Int64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (sum = sum + d)
│         (@_5 = Base.iterate(%2, %9))
│   %12 = (@_5 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return sum



In [42]:
@code_warntype arith_series(1, 2.0, 4.0)

MethodInstance for arith_series(::Int64, ::Float64, ::Float64)
  from arith_series(a1, d, n) in Main at In[35]:8
Arguments
  #self#::Core.Const(arith_series)
  a1::Int64
  d::Float64
  n::Float64
Locals
  @_5::Union{Nothing, Tuple{Float64, Int64}}
  sum::Union{Float64, Int64}
  i::Float64
Body::Union{Float64, Int64}
1 ─       (sum = a1)
│   %2  = Main.range(0, n)::StepRangeLen{Float64, Base.TwicePrecision{Float64}, Base.TwicePrecision{Float64}, Int64}
│         (@_5 = Base.iterate(%2))
│   %4  = (@_5 === nothing)::Bool
│   %5  = Base.not_int(%4)::Bool
└──       goto #4 if not %5
2 ┄ %7  = @_5::Tuple{Float64, Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (sum = sum + d)
│         (@_5 = Base.iterate(%2, %9))
│   %12 = (@_5 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return sum



In [43]:
@code_warntype comp_arith_series(1, 2, 4, 20)

MethodInstance for comp_arith_series(::Int64, ::Int64, ::Int64, ::Int64)
  from comp_arith_series(a1, d, n, comp) in Main at In[37]:9
Arguments
  #self#::Core.Const(comp_arith_series)
  a1::Int64
  d::Int64
  n::Int64
  comp::Int64
Locals
  sum::Int64
Body::Int64
1 ─      (sum = Main.arith_series(a1, d, n))
│   %2 = (sum >= comp)::Bool
└──      goto #3 if not %2
2 ─      return sum
3 ─      return comp



In [44]:
@code_warntype comp_arith_series(1.0, 2.0, 4, 20)

MethodInstance for comp_arith_series(::Float64, ::Float64, ::Int64, ::Int64)
  from comp_arith_series(a1, d, n, comp) in Main at In[37]:9
Arguments
  #self#::Core.Const(comp_arith_series)
  a1::Float64
  d::Float64
  n::Int64
  comp::Int64
Locals
  sum::Float64
Body::Union{Float64, Int64}
1 ─      (sum = Main.arith_series(a1, d, n))
│   %2 = (sum >= comp)::Bool
└──      goto #3 if not %2
2 ─      return sum
3 ─      return comp



The result of the ``@code_warntype`` confirms that if all the data type of the input values does not match the data type of the return values then we would receive a possible source of error as shown by the red text. This also applies when we have two return values with different data types. However, only a warning is given as shown by the yellow text when another input for the data type which does not directly affect the calculation just like that of ``n``.

Since we now have an idea of the possible sources of error, we write a new version where the issues are fixed. Instead of a for-loop, we implement the arithmetic series using Eq. \eqref{1}. This will work since Julia can choose the data type for the result and what we now have is a one liner code. This is unlike the function with the type-instability where the lines of the code have different data types.

In [45]:
"""
    fixed_arith_series(a1, d, n)
Stable version of arith_series(a1, d, n)
"""
function fixed_arith_series(a1, d, n)
    return a1+d*n   # Equation 1
end

fixed_arith_series

In [46]:
@code_warntype fixed_arith_series(1, 2.0, 4)

MethodInstance for fixed_arith_series(::Int64, ::Float64, ::Int64)
  from fixed_arith_series(a1, d, n) in Main at In[45]:5
Arguments
  #self#::Core.Const(fixed_arith_series)
  a1::Int64
  d::Float64
  n::Int64
Body::Float64
1 ─ %1 = (d * n)::Float64
│   %2 = (a1 + %1)::Float64
└──      return %2



In [47]:
@code_warntype fixed_arith_series(1, 2, 4)

MethodInstance for fixed_arith_series(::Int64, ::Int64, ::Int64)
  from fixed_arith_series(a1, d, n) in Main at In[45]:5
Arguments
  #self#::Core.Const(fixed_arith_series)
  a1::Int64
  d::Int64
  n::Int64
Body::Int64
1 ─ %1 = (d * n)::Int64
│   %2 = (a1 + %1)::Int64
└──      return %2



Finally, we also defined a stable version of the code which compares ``comp`` with the result of the arithmetic series. We do this by converting the data types of the return values to always be ``Float64`` by using ``convert()``.

In [48]:
"""
    fcomp_fixed_arith_series(a1, d, n, comp)
Stable version of comp_arith_series(a1, d, n, comp)
"""
function fcomp_arith_series(a1, d, n, comp)
    sum = convert(Float64,fixed_arith_series(a1, d, n))   # Convert the result for Arithmetic series as float.
    if sum >= comp
        return sum
    else
        return convert(Float64, comp)           # Convert the datatype of comp to Float64
    end
end

fcomp_arith_series

In [49]:
@code_warntype fcomp_arith_series(1.0, 2.0, 4, 20)

MethodInstance for fcomp_arith_series(::Float64, ::Float64, ::Int64, ::Int64)
  from fcomp_arith_series(a1, d, n, comp) in Main at In[48]:5
Arguments
  #self#::Core.Const(fcomp_arith_series)
  a1::Float64
  d::Float64
  n::Int64
  comp::Int64
Locals
  sum::Float64
Body::Float64
1 ─ %1 = Main.fixed_arith_series(a1, d, n)::Float64
│        (sum = Main.convert(Main.Float64, %1))
│   %3 = (sum >= comp)::Bool
└──      goto #3 if not %3
2 ─      return sum
3 ─ %6 = Main.convert(Main.Float64, comp)::Float64
└──      return %6



There are other ways to define a stable function. It is important to first determine the data types of the initial and return values. This is implemented by using ``typeof()`` and ``eltype()``. While we set our data types to be floating numbers for the return values using ``convert()``, we can also do this by multiplying the expression by ``1.0``.

# KR6
Demonstration of how Arrays containing ambiguous/abstract types often results to slow execution of codes. The BenchmarkTools may be useful in this part.

In [50]:
using BenchmarkTools

In [51]:
"""
    my_sum(x::Array{T}) where T <: Number
Returns the sum of all the elements in Array x.
"""
function my_sum(x::Array{T}) where T <: Number
    sum = zero(T)   # Initialise the sum
    for i in 1:length(x)
        sum += x[i]
    end
    return sum
end

my_sum

To differentiate the execution time based on the ambiguous/abstract types, we define an array of ``Float64`` and ``Number``.

In [52]:
x1 = Float64[ 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0];
x2 = Number[ 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 11.0, 12.0, 13.0, 14.0, 15.0];

Next, we implement ``@benchmark`` to record the execution time.

In [53]:
statx1 = @benchmark my_sum(x1)

BenchmarkTools.Trial: 10000 samples with 996 evaluations.
 Range (min … max):  23.301 ns …  1.828 μs  ┊ GC (min … max): 0.00% … 98.46%
 Time  (median):     24.683 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   26.161 ns ± 29.289 ns  ┊ GC (mean ± σ):  1.91% ±  1.70%

  ▃▁ ▂▅█▅▅▄▆▄▃▃▂▂▁▁▁                                          ▂
  ██▅██████████████████████▇▇████▇█▇▇▇███▇▇▇▆▇▆▆▄▄▆▄▃▄▄▅▅▁▄▃▄ █
  23.3 ns      Histogram: log(frequency) by time      36.6 ns <

 Memory estimate: 16 bytes, allocs estimate: 1.

In [54]:
statx2 = @benchmark my_sum(x2)

BenchmarkTools.Trial: 10000 samples with 550 evaluations.
 Range (min … max):  208.484 ns …   3.464 μs  ┊ GC (min … max): 0.00% … 93.47%
 Time  (median):     213.258 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   225.395 ns ± 155.298 ns  ┊ GC (mean ± σ):  3.62% ±  4.91%

  ▅▅▇█▄▄▅▃▄▃▂▂▂▁▁                                               ▂
  ██████████████████▇██▇▇▇▅▆▇▆▆▆▆▆▇▆▇▆▅▆▆▆▇▆▆▅▄▄▄▁▃▃▃▁▅▄▄▁▅▄▄▃▅ █
  208 ns        Histogram: log(frequency) by time        301 ns <

 Memory estimate: 240 bytes, allocs estimate: 15.

In [55]:
println(" The ratio for the median time of a Number array and an abstract Float64 array is $(median(statx2.times)/median(statx1.times))")

 The ratio for the median time of a Number array and an abstract Float64 array is 8.639975150134603


Hence, a more specific array will result to faster execution of the code. In this case, the Float64 Array is $\approx 8$ times faster than the Number Array.